In [1]:
import os

import cv2 as cv
from PIL import Image

import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


In [36]:
root_path = "ROOT_PATH_TO_FOLDER<sw_data>"

filenames_Beech = next(
    os.walk(f"{root_path}sw_data/train_test_sw/train_sw/Beech"), (None, None, [])
)[2]
filenames_Gardenia = next(
    os.walk(f"{root_path}sw_data/train_test_sw/train_sw/Gardenia"), (None, None, [])
)[2]
filenames_Lemon = next(
    os.walk(f"{root_path}sw_data/train_test_sw/train_sw/Lemon"), (None, None, [])
)[2]
filenames_Mean = next(
    os.walk(f"{root_path}sw_data/train_test_sw/train_sw/Mean"), (None, None, [])
)[2]
filenames_Tomato = next(
    os.walk(f"{root_path}sw_data/train_test_sw/train_sw/Tomato"), (None, None, [])
)[2]

len(filenames_Tomato)


206

In [208]:
def modifyImage(filenames_name, class_name, root_path, save_root_path):
    for filenames in filenames_name:
        img = cv.imread(f"{root_path}/{class_name}/{filenames}")
        filenames = filenames.replace("frame", "frameModifed")
        laplacian = cv.Laplacian(img, cv.CV_64F)
        im = Image.fromarray((laplacian * 255).astype(np.uint8))
        im = im.convert("RGB", palette=Image.ADAPTIVE, colors=8)
        im.save(f"{save_root_path}/{class_name}/{filenames}")


In [207]:
save_dir = f"{root_path}sw_data/train_test_sw/train_sw/"

modifyImage(filenames_Beech, "Beech", root_path + "train_sw/", save_dir)
modifyImage(filenames_Gardenia, "Gardenia", root_path + "train_sw/", save_dir)
modifyImage(filenames_Lemon, "Lemon", root_path + "train_sw/", save_dir)
modifyImage(filenames_Mean, "Mean", root_path + "train_sw/", save_dir)
modifyImage(filenames_Tomato, "Tomato", root_path + "train_sw/", save_dir)


Beech1frame0000012.png
Beech1frameModifed0000012.png
Flower3frame0000012.png
Flower3frameModifed0000012.png
Lemon4frame0000012.png
Lemon4frameModifed0000012.png
Mean6frame0000012.png
Mean6frameModifed0000012.png
Tomato5frame0000012.png
Tomato5frameModifed0000012.png


In [25]:
dir = f"{root_path}sw_data/train_test_sw/train_sw/"
print(os.listdir(root_path))
os.listdir(save_dir)


['Beech', 'Gardenia', 'Lemon', 'Mean', 'Tomato']


['Beech', '.ipynb_checkpoints', 'Tomato', 'Mean', 'Lemon', 'Gardenia']

In [177]:
test_labels_path = f"{root_path}sw_data/test_sw/test_labels.json"
test_labels = pd.read_json(test_labels_path)
test_labels["path"] = test_labels.filename.map(
    lambda filename: f"{root_path}test_sw/{filename}"
)
test_labels


,filename,value,path
0,Beech1frame0000022.png,Beech,/content/drive/MyDrive/sw_data/train_test_sw/t...
1,Beech1frame0000026.png,Beech,/content/drive/MyDrive/sw_data/train_test_sw/t...
2,Beech1frame0000027.png,Beech,/content/drive/MyDrive/sw_data/train_test_sw/t...
3,Beech1frame0000044.png,Beech,/content/drive/MyDrive/sw_data/train_test_sw/t...
4,Beech1frame0000045.png,Beech,/content/drive/MyDrive/sw_data/train_test_sw/t...
...,...,...,...
254,Tomato5frame0000300.png,Tomato,/content/drive/MyDrive/sw_data/train_test_sw/t...
255,Tomato5frame0000309.png,Tomato,/content/drive/MyDrive/sw_data/train_test_sw/t...
256,Tomato5frame0000311.png,Tomato,/content/drive/MyDrive/sw_data/train_test_sw/t...
257,Tomato5frame0000313.png,Tomato,/content/drive/MyDrive/sw_data/train_test_sw/t...


In [130]:
train_dirs = os.listdir(f"{root_path}sw_data/train_sw")

train_labels = pd.DataFrame()

for d in train_dirs:
    for f in os.listdir(f"{root_path}sw_data/train_sw/{d}"):
        df_dict = pd.DataFrame(
            [
                {
                    "filename": f,
                    "path": f"{root_path}sw_data/train_sw/{d}/{f}",
                    "value": d,
                }
            ]
        )
        # print(df_dict)
        train_labels = pd.concat([train_labels, df_dict], ignore_index=True)

train_labels


,filename,path,value
0,Beech1frame0000012.png,/content/drive/MyDrive/sw_data/train_test_sw/t...,Beech
1,Beech1frame0000013.png,/content/drive/MyDrive/sw_data/train_test_sw/t...,Beech
2,Beech1frame0000014.png,/content/drive/MyDrive/sw_data/train_test_sw/t...,Beech
3,Beech1frame0000015.png,/content/drive/MyDrive/sw_data/train_test_sw/t...,Beech
4,Beech1frame0000016.png,/content/drive/MyDrive/sw_data/train_test_sw/t...,Beech
...,...,...,...
2049,Tomato5frameModifed0000312_filter.png,/content/drive/MyDrive/sw_data/train_test_sw/t...,Tomato
2050,Tomato5frameModifed0000315_filter.png,/content/drive/MyDrive/sw_data/train_test_sw/t...,Tomato
2051,Tomato5frameModifed0000316_filter.png,/content/drive/MyDrive/sw_data/train_test_sw/t...,Tomato
2052,Tomato5frameModifed0000317_filter.png,/content/drive/MyDrive/sw_data/train_test_sw/t...,Tomato


In [88]:
# Define the model
class MLP(torch.nn.Module):
    def __init__(self, input_shape):
        # Initialize super class
        super().__init__()

        # Build model using Sequential container
        self.model = nn.Sequential(
            # Add input layer
            nn.Linear(input_shape, 512),
            # Add ReLU activation
            nn.ReLU(),
            # Add Another layer
            nn.Linear(512, 256),
            # Add ReLU activation
            nn.ReLU(),
            # Add Output layer
            nn.Linear(256, 5),
        )

    def forward(self, x):
        # Forward pass
        return self.model(x)


In [201]:
def train():
    model.train()
    running_loss = 0
    running_correct = 0

    for i, (x_train, y_train) in enumerate(train_loader):
        if not (i + 1) % 10:
            print(f"Step {i+1} completed")
        # Forward pass:
        # Flatten the image since the input to the network
        # x_train = transforms.ToTensor(x_train)
        x_train = x_train.view(x_train.shape[0], -1)
        # Compute predicted y by passing x to the model
        y = model(x_train)

        # Compute and print loss
        # print("there is a Y:", y)
        # print("There is a y_train:", y_train)
        loss = loss_function(y, torch.tensor(y_train))
        running_loss += loss.item()

        # Compute Accuracy
        y_pred = y.argmax(dim=1)
        correct = torch.sum(y_pred == y_train)
        running_correct += correct

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()

        # Calculate gradient using backward pass
        loss.backward()

        # Update model parameters (weights)
        optimizer.step()

    return running_loss / len(train_loader), running_correct.item() / len(
        train_loader.dataset
    )


In [192]:
def val():
    model.eval()
    running_loss = 0
    running_correct = 0
    with torch.no_grad():
        for (x_val, y_val) in validation_loader:

            # Forward pass:
            # Flatten the image since the input to the network is a 784 dimensional vector
            x_val = x_val.view(x_val.shape[0], -1)

            # Compute raw score by passing x to the model
            y = model(x_val)

            # Score to probability using softmax
            prob = nn.functional.softmax(y, dim=1)

            # Compute Accuracy
            y_pred = prob.argmax(dim=1)

            correct = torch.sum(y_pred == y_val)
            running_correct += correct

            # Compute and print loss
            loss = loss_function(y, y_val)
            running_loss += loss.item()

    return running_loss / len(validation_loader), running_correct.item() / len(
        validation_loader.dataset
    )


In [183]:
class CustomImageDataset(Dataset):
    def __init__(self, df, transform=None, target_transform=None):
        self.img_labels = df["value"]
        self.img_dir = df["path"]
        self.transform = transform
        self.target_transform = target_transform
        self.labels_dict = {label: i for i, label in enumerate(df["value"].unique())}

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_dir[idx]
        image = Image.open(img_path)
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image[:3], self.labels_dict[label]


In [193]:
# Batch size : How many images are used to calculate the gradient
batch_size = 32

# Train DataLoader
train_dataset = CustomImageDataset(
    train_labels,
    transform=transforms.Compose(
        [
            transforms.Resize(64),
            # transforms.Normalize(),
            transforms.ToTensor(),
        ]
    ),
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
# Validation DataLoader
validation_dataset = CustomImageDataset(
    test_labels,
    transform=transforms.Compose(
        [
            transforms.Resize(64),
            # transforms.Normalize(),
            transforms.ToTensor(),
        ]
    ),
)
validation_loader = DataLoader(
    dataset=validation_dataset, batch_size=batch_size, shuffle=False
)


In [194]:
train_dataset[0][0], train_dataset[0][0].shape


(tensor([[[0.0627, 0.0745, 0.0824,  ..., 0.0745, 0.0667, 0.0510],
          [0.0667, 0.0784, 0.0863,  ..., 0.0588, 0.0510, 0.0549],
          [0.0824, 0.0902, 0.0941,  ..., 0.0510, 0.0549, 0.0588],
          ...,
          [0.0549, 0.0588, 0.0627,  ..., 0.0471, 0.0510, 0.0549],
          [0.0549, 0.0549, 0.0549,  ..., 0.0549, 0.0549, 0.0510],
          [0.0588, 0.0549, 0.0549,  ..., 0.0627, 0.0510, 0.0471]],
 
         [[0.0667, 0.0784, 0.0784,  ..., 0.0784, 0.0667, 0.0549],
          [0.0667, 0.0784, 0.0863,  ..., 0.0588, 0.0510, 0.0549],
          [0.0863, 0.0902, 0.0941,  ..., 0.0549, 0.0549, 0.0549],
          ...,
          [0.0588, 0.0627, 0.0667,  ..., 0.0471, 0.0510, 0.0549],
          [0.0588, 0.0549, 0.0549,  ..., 0.0549, 0.0588, 0.0510],
          [0.0627, 0.0588, 0.0588,  ..., 0.0627, 0.0510, 0.0431]],
 
         [[0.0471, 0.0549, 0.0588,  ..., 0.0549, 0.0471, 0.0392],
          [0.0471, 0.0588, 0.0627,  ..., 0.0392, 0.0353, 0.0392],
          [0.0588, 0.0667, 0.0667,  ...,

In [202]:
# Training parameters
num_epochs = 20

# Construct model
model = MLP(np.prod(train_dataset[0][0].shape))

# Define loss function
loss_function = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=1e-3)

print(model)


MLP(
  (model): Sequential(
    (0): Linear(in_features=12288, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=5, bias=True)
  )
)


In [203]:
train_loss_history = []
val_loss_history = []
train_acc_history = []
val_acc_history = []

print("Starting Training...")
for ep in range(num_epochs):
    train_loss, train_acc = train()
    val_loss, val_acc = val()
    print(
        "Epoch: {}, Train Loss = {:.3f}, Train Acc = {:.3f} , Val Loss = {:.3f}, Val Acc = {:.3f}".format(
            ep + 1, train_loss, train_acc, val_loss, val_acc
        )
    )
    train_loss_history.append(train_loss)
    val_loss_history.append(val_loss)
    train_acc_history.append(train_acc)
    val_acc_history.append(val_acc)


Starting Training...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Step 10 completed
Step 20 completed
Step 30 completed
Step 40 completed
Step 50 completed
Step 60 completed
Epoch: 0, Train Loss = 1.611, Train Acc = 0.198 , Val Loss = 1.611, Val Acc = 0.201
Step 10 completed
Step 20 completed
Step 30 completed
Step 40 completed
Step 50 completed
Step 60 completed
Epoch: 1, Train Loss = 1.611, Train Acc = 0.199 , Val Loss = 1.611, Val Acc = 0.205
Step 10 completed
Step 20 completed
Step 30 completed
Step 40 completed
Step 50 completed
Step 60 completed
Epoch: 2, Train Loss = 1.610, Train Acc = 0.202 , Val Loss = 1.611, Val Acc = 0.205
Step 10 completed
Step 20 completed
Step 30 completed
Step 40 completed
Step 50 completed
Step 60 completed
Epoch: 3, Train Loss = 1.610, Train Acc = 0.202 , Val Loss = 1.610, Val Acc = 0.205
Step 10 completed
Step 20 completed
Step 30 completed
Step 40 completed
Step 50 completed
Step 60 completed
Epoch: 4, Train Loss = 1.609, Train Acc = 0.204 , Val Loss = 1.610, Val Acc = 0.212
Step 10 completed
Step 20 completed
Step